# To send personalized table as attachment to ADM vendors

### SOP for using this notebook to send personalized table from a dump file as attachment to ADM vendors
1. Go to https://myaccount.google.com/apppasswords and create new app profile & password associated with your personal flipkart account. Ensure you have 2 step authorization with your flipkart account.
2. The password has to be updated in Step 5 along with your email ID
3. The updated Vendor list along with their email IDs has to be stored in an .xlsx file locally & the path to the file has to be updated in Step 2.
4. The respective vendor files will be processed in a local folder for processing. The folder path has to be updated in Step 4.
5. The file has to be executed in the order mentioned in this file from Step 1 to Step 5.
6. The file contains paths used during testing. Please update before using.

## 1. Install necessary packages in python

In [1]:
# Install packages
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta
import pandas as pd
import seaborn as sns
import requests
import os
from pathlib import Path

#for sending mail
import smtplib
#import imghdr
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import glob

## 2. Read Vendor names & their email IDs from local excel file

In [2]:
df = pd.read_csv(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\ODH Model pincode Tracker.csv')
df1 = df[['ODH/MDH Name','Type','Vendor','Zone','State']]
Map = df1.drop_duplicates()
Map.rename(columns={'Vendor':'Vendor Name'},inplace=True)
Map['Vendor Name'] = Map['Vendor Name'].astype(str).str.upper()
Map['ODH/MDH Name'] = Map['ODH/MDH Name'].astype(str).str.upper()
Map

,ODH/MDH Name,Type,Vendor Name,Zone,State
0,SASTHURAVOORODH_THO,ODH,SAS,SOUTH,KERALA
1,RJSCIMPHALODH_IMP,ODH,RJSC,EAST,MANIPUR
2,IMPHALSPLITMDH_IMP,MDH in Bin,RJSC,EAST,MANIPUR
3,ELASTICRUNBHAWANIPURODH_KOL,ODH,ELASTICRUN,EAST,WEST BENGAL
4,CABTGARIAHATODH_KOL,ODH,LOADSHARE,EAST,WEST BENGAL
...,...,...,...,...,...
2393,UCCMDH_SHL,MDH,LIGHTWINGS,EAST,MEGHALAYA
2394,JHALDAHUB_JAA,ODH,LOADSHARE,EAST,WEST BENGAL
2395,GANGADRAMDH_KDO,MDH,SHADOWFAX,WEST,GUJARAT
2396,MAWSYNRAMMDH_SHL,MDH,LIGHTWINGS,EAST,MEGHALAYA


In [4]:
# Read each file separately with explicit file paths
FkFraud = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\FKFraud01A22A.xlsx", skiprows=2)
MPFBF_Fraud = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\FKMPFBF01A21A.xlsx", skiprows=2)
SPFFK = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\FKSPF1A22A.xlsx", skiprows=2)
Prexo = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\Prexo01A19A.xlsx", skiprows=2)
MyntraRVP = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\MyntraQ223A.xlsx', engine="openpyxl", sheet_name="RVP DATA")
MyntraRTO = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\MyntraQ223A.xlsx', engine="openpyxl", sheet_name="RTO DATA")
SPFMyntra = pd.read_excel(r"C:\Users\prajwal.d1\Desktop\Fraud_Data\MyntraSPF1M20M.xlsx") #NM



In [5]:
FkFraud.columns

Index(['Value (Cr)', 'Source.Name', 'Remarks', 'Final Amount', 'BSM',
       'return_amount', 'warehouse_id', 'shipment_id', 'pv_id', 'T/F',
       ...
       '3PL Flag', 'MH Flag', 'Buck', 'DH', 'Month', 'Asset', 'Week',
       'MP Null Flag', 'MP 3PL Flag', 'MP MH Flag'],
      dtype='object', length=123)

In [6]:
FkFraud['last_dh_received_name'] = FkFraud['last_dh_received_name'].astype(str).str.upper()
MPFBF_Fraud['last_dh_received_name'] = MPFBF_Fraud['last_dh_received_name'].astype(str).str.upper()
SPFFK['Final DH Name'] = SPFFK['Final DH Name'].astype(str).str.upper()
Prexo['last_dh_received_name'] = Prexo['last_dh_received_name'].astype(str).str.upper()
MyntraRVP['Hub Name'] = MyntraRVP['Hub Name'].astype(str).str.upper()
MyntraRTO['Hub Name '] = MyntraRTO['Hub Name '].astype(str).str.upper()
SPFMyntra ['ek_hub'] = SPFMyntra['ek_hub'].astype(str).str.upper()



In [7]:
FkFraud = pd.merge(FkFraud, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'last_dh_received_name', right_on = 'ODH/MDH Name')
FkFraud = FkFraud.drop_duplicates()
FkFraud.drop(columns = ['ODH/MDH Name'],inplace = True)

MPFBF_Fraud = pd.merge(MPFBF_Fraud, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'last_dh_received_name', right_on = 'ODH/MDH Name')
MPFBF_Fraud=MPFBF_Fraud.drop_duplicates()
MPFBF_Fraud.drop(columns = ['ODH/MDH Name'],inplace = True)

SPFFK = pd.merge(SPFFK, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'Final DH Name', right_on = 'ODH/MDH Name')
SPFFK=SPFFK.drop_duplicates()
SPFFK.drop(columns = ['ODH/MDH Name'],inplace = True)

Prexo = pd.merge(Prexo, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'last_dh_received_name', right_on = 'ODH/MDH Name')
Prexo=Prexo.drop_duplicates()
Prexo.drop(columns = ['ODH/MDH Name'],inplace = True)

MyntraRVP = pd.merge(MyntraRVP, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'Hub Name', right_on = 'ODH/MDH Name')
MyntraRVP = MyntraRVP.drop_duplicates()
MyntraRVP.drop(columns = ['ODH/MDH Name'],inplace = True)

MyntraRTO = pd.merge(MyntraRTO, Map[['ODH/MDH Name','Vendor Name']], how='left', left_on = 'Hub Name ', right_on = 'ODH/MDH Name')
MyntraRTO = MyntraRTO.drop_duplicates()
MyntraRTO.drop(columns = ['ODH/MDH Name'],inplace = True)






In [8]:
# Reading a database of vendors & email ids
VendorContact = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Vendor_Mail.xlsx', dtype=str)

In [9]:
VendorNames = VendorContact["Vendor Name"].tolist()
VendorMail = VendorContact["Vendor Mail"].tolist()

## 3. Creating file directory & clear old data

In [10]:
# Create a folder and update path here
DownloadPath = "C:\\Users\\prajwal.d1\\Desktop\\Fraud_Data\\Fraud_Reports_Files"

In [11]:
# Remove all files in the specified folder
os.chdir(f"{DownloadPath}")
all_files = os.listdir()

for f in all_files:
    os.remove(f)

## 4. Processing data for respective vendors

In [12]:
# Create separate files for vendors
attachment_filename_fwd = []

for x in range(len(VendorNames)):
    VendorData_FkFraud = FkFraud.loc[FkFraud['Vendor Name'] == VendorNames[x]]
    VendorData_MPFBF_Fraud = MPFBF_Fraud.loc[MPFBF_Fraud['Vendor Name'] == VendorNames[x]]
    VendorData_SPFFK = SPFFK.loc[SPFFK['Vendor Name'] == VendorNames[x]]
    VendorData_Prexo = Prexo.loc[Prexo['Vendor Name'] == VendorNames[x]]
    VendorData_MyntraRVP = MyntraRVP.loc[MyntraRVP['Vendor Name'] == VendorNames[x]]
    VendorData_MyntraRTO = MyntraRTO.loc[MyntraRTO['Vendor Name'] == VendorNames[x]]
    VendorData_SPFMyntra = SPFMyntra.loc[SPFMyntra['Vendor Name'] == VendorNames[x]]

    filename=VendorNames[x]+"_Fraud_Reports.xlsx"
    with pd.ExcelWriter(filename) as writer:
        VendorData_FkFraud.to_excel(writer, sheet_name='FK_Fraud_Final_1A22A',index=False)
        VendorData_MPFBF_Fraud.to_excel(writer, sheet_name='FKMPFBF_Final_1A21A',index=False)
        VendorData_SPFFK.to_excel(writer, sheet_name='SPFFK_Final_1A22A',index=False)
        VendorData_Prexo.to_excel(writer, sheet_name='Prexo_Final_01A19A',index=False)
        VendorData_MyntraRVP.to_excel(writer, sheet_name='MyntraRVP_Final_1A23A',index=False)
        VendorData_MyntraRTO.to_excel(writer, sheet_name='MyntraRTO_Final_1A23A',index=False)
        VendorData_SPFMyntra.to_excel(writer, sheet_name='SPF_Myntra_1M20M',index=False)
        
    attachment_filename_fwd.append(VendorNames[x]+"_Fraud_Reports.xlsx")


## 5. Send E-mail to vendor with their respective attachment

In [13]:
# Enter credentials of your account
EMAIL_ADDRESS = 'prajwal.d1@flipkart.com'
EMAIL_PASSWORD = 'ynvfvcvolkotfwbr'

In [14]:
for y in range(len(VendorNames)):
    msg = EmailMessage()
    msg['Subject'] = "Fraud Reports – 23 April'25 Part-1 "+VendorNames[y]
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = VendorMail[y]
    msg.set_content("""Dear Partner,

    Do find attached the various Fraud Data / OBD-CV Adh reports along with timelines for your reference. 
    Kindly review the details below and take necessary actions:
    
    1. FK Fraud:
    
   - Fraud cases under ₹2K have increased. 
     Please review the data at an OFD and Wishmaster level and share the RCA for these incidents.  


    2. Myntra Q2 RVP & RTO:
    
  -  There is a rise in cases of used products and missing brand tags. 
     Ensure Wishmasters (WMs) and Team Leads (TLs) are sensitized on this, and SmartChecks are strictly followed before picking up shipments.  
     

    3. Prexo:  
    
   - A high number of brand mismatches and fake mobile shipments are being picked up without processing through Yaantra. 
     TLs and WMs must adhere to Yaantra SmartChecks to prevent such issues.  


    4. SPF-FK & Myntra Q2:
    
   - A significant number of incorrect pickups are being done by WMs, and TLs are also not ensuring adherence to SmartChecks. 
     Immediate audits of pickups must be conducted.  
     

    5. MPFBF:
    
   - There is an increasing trend of wrong and fake pickups. 
     Please review TID-level data for high-value cases and take necessary measures.  
     

  

      Actionable Points:
          - Review the fraud reports and share your findings.  
          - Work on improving CV and OBD Non Adh at the Hub/FE level.
          
         
- NOTE: Issue warning letters to WMs where non-adherence is found in more than five shipments. Continued non-adherence will result in blacklisting from the system.  
    
Thanks & Regards,
Prajwal | 
7975770738""")
    attachment_filename_fwd=(VendorContact.loc[VendorContact['Vendor Name'] == VendorNames[y],"Vendor Name"]+"_Fraud_Reports.xlsx").tolist()
    files1=[attachment_filename_fwd[0]]
    for file in files1:
        with open(file,'rb') as f:
            file_data1 = f.read()
            file_name1 = f.name
        msg.add_attachment(file_data1, maintype='application', subtype='octet-stream', filename=file_name1)
    #PDF=Temp.loc[Temp['File']=='Gate_pass_Format.pdf',"File"].tolist()
    #files2 =[PDF[0]]
    #for file in files2:
     #   with open(file,'rb') as f:
      #      file_data2 = f.read()
       #     file_name2 = f.name
        #msg.add_attachment(file_data2, maintype='application', subtype='octet-stream', filename=file_name2)
    # with smtplib.SMTP('smtp.gmail.com', 587, timeout=30) as smtp:
     # smtplib.SMTP('smtp.gmail.com', 587, timeout=60) as smtp:
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp: 
        smtp.login(EMAIL_ADDRESS,EMAIL_PASSWORD)
        smtp.send_message(msg)    

In [15]:
#FwdDump.head()

In [ ]:
# Remove all files in the specified folder
os.chdir(f"{DownloadPath}")
all_files = os.listdir()
os.getcwd()

In [ ]:
(VendorContact.loc[VendorContact['Vendor Name'] == VendorNames[y],"Vendor Name"]+"_RTO_RVP_UT_28.xlsx")

In [ ]:
#   10. Relabeling Data : 1st Feb'25 to 03rd Feb'25
#       * There is a growing trend where fraudulent transactions are being relabeled as legitimate, which can significantly affect the accuracy of our fraud detection models.
#
#   11. Externalization Fraud loss - 1stJan'25 to 28thJan'25  
#       * There is High trend of DNU fraud is Externalization - Pl go through the TIDs and take action on WM Marking Undeli
#
#10. Externalization Fraud loss - 1stDec'24 to 31stDec'24  
#       * There is High trend of DNU fraud is Externalization - Pl go through the TIDs and take action on WM Marking Undel
#11. Externalization Fraud loss - 1stJan'25 to 07thJan'25  
#       * There is High trend of DNU fraud is Externalization - Pl go through the TIDs and take action on WM Marking Undel
